In [1]:
try:
	from sys import _getframe

except ImportError:
	import sys
	
	try:
		raise Exception
	
	except:
		if (
			not hasattr(sys.exc_info()[2], 'tb_frame') or
			not hasattr(sys.exc_info()[2].tb_frame, 'f_back')
		):
			raise ImportError(
				"Unable to capture frames. sys._getframe() is not supported "
				"in this Python implementation, and the traceback object does "
				"not conform to CPython specifications."
			)
		
		else:
			def _getframe(level=0):
				"""
				A reimplementation of `sys._getframe()`. `level` is the number
				of levels deep into the stack to grab the frame from
				(default: 0).
				
				`_getframe()` is a private function, and isn't guaranteed to
				exist in all versions and implementations of Python. This
				function is about 2x slower.
				
				`sys.exc_info()` only returns helpful information if an
				exception has been raised.
				
				"""
				
				try:
					raise Exception
				
				except:
					# sys.exc_info() returns (type, value, traceback).
					frame = sys.exc_info()[2].tb_frame
					
					# + 1 to account for our exception.
					for i in xrange(0, level + 1):
						frame = frame.f_back
				
				finally:
					sys.exc_clear()
				
				return frame
	
	finally:
		sys.exc_clear()


class TailCallSigil(Exception):
	def __init__(self, args, kwargs):
		self.args = args
		self.kwargs = kwargs


def tailcall(function):
	"""
	A decorator for functions set up for tail call optimization. If your
	function *isn't* set up for tail call optimization, this won't work
	as intended.
	
	You should probably never use this.
	
	(Mutually recursive functions work, so long as all functions have the
	`@tail_call` decorator.)
	
	"""
	
	def wrapper(*args, **kwargs):
		"""
		Wraps a function optimized for tail calls, allowing them to reuse the
		stack.
		
		"""
		
		try:
			# Check to make sure we aren't our own grandparent.
			frame_0, frame_2 = _getframe(0), _getframe(2)
			
			if frame_2 and frame_0.f_code == frame_2.f_code:
				raise TailCallSigil(args, kwargs)
		
		except ValueError:
			pass
		
		while True:
			try:
				# Will be called decorated, hence the grandparents above.
				result = function(*args, **kwargs)
			
			except TailCallSigil as sigil:
				args, kwargs = sigil.args, sigil.kwargs
			
			else:
				return result
	
	return wrapper

In [19]:
def factor1(x):  
    if x == 0:  
        return 1 
    else:  
        return x * factor1(x - 1) 

In [20]:
def factor2(x):  
    result = 1 
    i = 2 
    while i <= x:  
        result = result * i  
        i = i + 1 
    return result

In [21]:
def factor3(x):  
    res = 1 
    for i in xrange(2, x + 1):  
        res *= i  
    return res

In [22]:
factor4 = lambda x: x and x * factor4(x - 1) or 1

In [34]:
factor5 = lambda x: reduce(int.__mul__, xrange(2, x + 1), 1)

In [52]:
import sys  
# @tailcall 
def factor6(x, acc=1):  
    if x: return factor6(x.__sub__(1), acc.__mul__(x))  
    return acc

In [25]:
import os  
def factor8(x):  
    return os.system('factorial ' + str(x))  

In [44]:
print factor1(12)
print factor2(12)
print factor3(12)
print factor4(12)
print factor5(12)
print factor6(12)
print factor8(12)

479001600
479001600
479001600
479001600
479001600
479001600
1


In [45]:
%timeit factor1(12)

1000000 loops, best of 3: 1.66 µs per loop


In [46]:
%timeit factor2(12)

1000000 loops, best of 3: 921 ns per loop


In [47]:
%timeit factor3(12)

1000000 loops, best of 3: 852 ns per loop


In [48]:
%timeit factor4(12)

1000000 loops, best of 3: 1.75 µs per loop


In [49]:
%timeit factor5(12)

1000000 loops, best of 3: 1.81 µs per loop


In [53]:
%timeit factor6(12)

100000 loops, best of 3: 5.08 µs per loop


In [51]:
%timeit factor8(12)

10 loops, best of 3: 30 ms per loop
